In [1]:
import logging
from pathlib import Path
from typing import Iterable, Optional

import datasets
import numpy as np
import pandas as pd
import torch
import typer
import yaml
from gluonts.dataset.split import split
from gluonts.ev.metrics import MASE, MeanWeightedSumQuantileLoss
from gluonts.itertools import batcher
from gluonts.model.evaluation import evaluate_forecasts
from gluonts.model.forecast import SampleForecast
from tqdm.auto import tqdm

from chronos import ChronosPipeline

app = typer.Typer(pretty_exceptions_enable=False)
from evaluate import *

/home/ubuntu/chronos/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/chronos/venv/lib/python3.10/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [2]:
def load_and_split_dataset(backtest_config: dict):
    hf_repo = backtest_config["hf_repo"]
    dataset_name = backtest_config["name"]
    offset = backtest_config["offset"]
    prediction_length = backtest_config["prediction_length"]
    num_rolls = backtest_config["num_rolls"]

    # This is needed because the datasets in autogluon/chronos_datasets_extra cannot
    # be distribued due to license restrictions and must be generated on the fly
    trust_remote_code = True if hf_repo == "autogluon/chronos_datasets_extra" else False

    ds = datasets.load_dataset(
        hf_repo, dataset_name, split="train", trust_remote_code=trust_remote_code
    )
    ds.set_format("numpy")

    gts_dataset = to_gluonts_univariate(ds)

    # Split dataset for evaluation
    _, test_template = split(gts_dataset, offset=offset)
    test_data = test_template.generate_instances(prediction_length, windows=num_rolls)

    return test_data

In [33]:
with open('./chronos-forecasting/scripts/evaluation/configs/') as fp:
    config = yaml.safe_load(fp)
test_data = load_and_split_dataset(backtest_config=config[0])

In [34]:
test_data

TestData(dataset=[{'start': Period('2020-07-22', 'D'), 'target': array([68., 68., 76., ...,  0.,  0.,  0.], dtype=float32)}, {'start': Period('2020-07-22', 'D'), 'target': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}, {'start': Period('2020-07-22', 'D'), 'target': array([0., 2., 1., ..., 0., 0., 0.], dtype=float32)}, {'start': Period('2020-07-22', 'D'), 'target': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}, {'start': Period('2020-07-22', 'D'), 'target': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}, {'start': Period('2020-07-22', 'D'), 'target': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}, {'start': Period('2020-07-22', 'D'), 'target': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}, {'start': Period('2020-07-22', 'D'), 'target': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}, {'start': Period('2020-07-22', 'D'), 'target': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}, {'start': Period('2020-07-22', 'D'), 'target': array([ 6.,  8., 10.

In [26]:
!pwd

/home/ubuntu/.jupyter
